In [ ]:
def infogain(P,N):
    import math
    return -P/(P+N)*math.log2(P/(P+N)) - N/(P+N)*math.log2(N/(P+N))

In [ ]:
def insertNode(tree,addTo,Node):
    for k,v in tree.items():
        if isinstance(v,dict):
            tree[k]=insertNode(v,addTo,Node)
        if addTo in tree:
            if isinstance(tree[addTo],dict):
                tree[addTo][Node]='None'
            else:
                tree[addTo]={Node:'None'}
    return tree        

In [ ]:
def insertConcept(tree,addTo,Node):
    for k,v in tree.items():
        if isinstance(v,dict):
            tree[k]=insertConcept(v,addTo,Node)
        if addTo in tree:
            tree[addTo]=Node
    return tree

In [ ]:
def getNextNode(data,attribute_list,concept,conceptVals,tree,addTo):
    total=data.shape[0]
    if total==0:
        return tree
    countC={}
    for cval in conceptVals:
        dataCC=data[data[concept]==cval]
        countC[cval]=dataCC.shape[0]

    if countC[conceptVals[0]]==0:
        tree=insertConcept(tree,addTo,conceptVals[1])
        return tree
    if countC[conceptVals[1]]==1:
        tree=insertConcept(tree,addTo,conceptVals[0])
        return tree
    
    classEntropy=countC[conceptVals[0]] + countC[conceptVals[1]]

    Attr={}
    for a in attribute_list:
        Attr[a]=list(set(data[a]))
    
    AttrCount={}
    EntropyAttr={}
    for att in Attr:
        for vals in Attr[att]:
            for c in conceptVals:
                iData=data[data[att]==vals]
                dataAtt=iData[iData[concept]==c]
                AttrCount[c]=dataAtt.shape[0]
            totalInfo=AttrCount[conceptVals[0]] + AttrCount[conceptVals[1]]
            if AttrCount[conceptVals[0]]==0 or AttrCount[conceptVals[1]]==0:
                infoGain=0
            else:
                infoGain=infogain(AttrCount[conceptVals[0]],AttrCount[conceptVals[1]])
            if att not in EntropyAttr:
                EntropyAttr[att]=(totalInfo/total)*infoGain
            else:
                EntropyAttr[att]=EntropyAttr[att] + (totalInfo/total)*infoGain
    gain={}
    for g in EntropyAttr:
        gain[g]=classEntropy - EntropyAttr[g]
    Node=max(gain,key=gain.get)
    tree=insertNode(tree,addTo,Node)

    for nD in Attr[Node]:
        tree=insertNode(tree,Node,nD)
        newData=data[data[Node]==nD].drop(Node,axis=1)
        attribute_list=list(newData)[:-1]
        tree=getNextNode(newData,attribute_list,concept,conceptVals,tree,nD)
    return tree

In [ ]:
import pandas as pd 
def main():
    data=pd.read_csv("D:\MyCode\Python\MLlab\dataset\heartDisease.csv")
    attribute_list=list(data)[:-1]
    concept= str(list(data)[-1])
    concept_vals=list(set(data[concept]))
    tree=getNextNode(data,attribute_list,concept,concept_vals,{'root':'None'},'root')
    return tree

In [ ]:
tree=main()['root']

In [ ]:
df=pd.read_csv("D:\MyCode\Python\MLlab\dataset\heartDisease.csv")
def test(tree,d):
    for k in tree:
        for v in tree[k]:
            if(d[k]==v and isinstance(tree[k][v],dict)):
                test(tree[k][v])
            elif(d[k]==v):
                print("classification : "+tree[k][v])
        


In [ ]:
print(tree)